### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Analysis of non-CO2 emissions from agriculture, forestry and land use

This notebook plots figures for emissions of CH4 and N2O from the land use sector
for **Figure 2.25** of the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
cats.remove('Above 2C')

In [ ]:
years = [2010, 2030, 2050, 2100]

In [ ]:
sr1p5.meta.rename(columns={'Kyoto-GHG|2010 (SAR)': 'kyoto_ghg_2010'}, inplace=True)

In [ ]:
df = sr1p5.filter(kyoto_ghg_2010='in range', category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
plot_ylabel = 'Global {} emissions from AFOLU ({})'
save_name = 'output/fig2.25{}.{}'
data_header = 'Emissions from AFOLU'

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

In [ ]:
def plotting_args(name, label, unit, panel_label=None, filetype='png'):
    return {'categories': cats, 'column': 'category', 'years': years,
            'add_marker': marker,
            'ylabel': plot_ylabel.format(label, unit),
            'save': save_name.format(name if panel_label is None else '{}_{}'.format(panel_label, name), filetype)}

In [ ]:
data = []

## Methane emissions from agriculture

In [ ]:
ch4_afolu = df.filter(variable='Emissions|CH4|AFOLU').timeseries()

In [ ]:
name = 'ch4_afolu'
species = 'CH4'
unit = 'MtCH4/yr'

_data = pyam.filter_by_meta(ch4_afolu, **filter_args)
boxplot_by_cat(_data, **plotting_args(name, species, unit, 'a'))
_data['species'] = species
data.append(_data)

## Nitrous oxide emissions from agriculture

In [ ]:
n2o_afolu = (
    df.filter(variable='Emissions|N2O|AFOLU')
    .convert_unit({'kt N2O/yr': ('Mt N2O/yr', 0.001)})
    .timeseries()
)

In [ ]:
name = 'no2_afolu'
species = 'N2O'
unit = 'MtN2O'

_data = pyam.filter_by_meta(n2o_afolu, **filter_args)
fig = boxplot_by_cat(_data, **plotting_args(name, species, unit, 'b'),
                     legend=False)
_data['species'] = species
data.append(_data)

## Export timeseries data to `xlsx`

In [ ]:
data = pd.concat(data).set_index(['species', 'category', 'marker'], append=True)

In [ ]:
data.head()

In [ ]:
data.reset_index().to_excel('output/fig2.25_data_table.xlsx')